In [1]:
import os

In [2]:
%pwd

'c:\\Text-Summarization\\research'

In [3]:
os.chdir("..\\")

In [4]:
%pwd

'c:\\Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer: str

In [6]:
import sys
sys.path.append("src")

In [7]:
%pip install -e src

import sys
from pathlib import Path

src_path = str((Path.cwd().parent / "src").resolve())
if src_path not in sys.path:
	sys.path.append(src_path)

from Text_Summarization.constants import *
from Text_Summarization.utils.common import read_yaml, create_dictionaries

Obtaining file:///C:/Text-Summarization/src
Note: you may need to restart the kernel to use updated packages.


ERROR: file:///C:/Text-Summarization/src does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from pathlib import Path
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_dictionaries([self.config.artifacts_root],verbose=True)

    def get_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_dictionaries([config.root_dir], verbose=True)
        
        data_transformation_config =  DataTransformationConfig(
            root_dir=Path(self.config.data_transformation.root_dir),
            data_path=Path(self.config.data_transformation.data_path),
            tokenizer=self.config.data_transformation.tokenizer
        )
        return data_transformation_config

In [10]:
import os
import logging
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [11]:
class DataTransformation:
    def __init__(self,config : DataTransformationConfig):
        self.config=config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer)

    def convert_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_id': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids'],
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_transformation_config()

    data_transformation = DataTransformation(config=data_transformation_config)
    status = data_transformation.convert()
except Exception as e:
    raise e

[2025-08-23 15:49:36,616] INFO - common - YAML file loaded successfully from config\config.yaml
[2025-08-23 15:49:36,617] INFO - common - YAML file loaded successfully from params.yaml
[2025-08-23 15:49:36,618] INFO - common - Created directory at artifacts
[2025-08-23 15:49:36,619] INFO - common - Created directory at artifacts/data_transformation


c:\Text-Summarization\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Prachi\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Text-Summarization\venv\Lib\site-packages\transformers\t